In [26]:
# Install dependencies if running in a fresh environment
!pip install -q pandas scikit-learn matplotlib seaborn ipywidgets

import pandas as pd
from sklearn.linear_model import LinearRegression
from google.colab import files
import os
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, Markdown

# Upload required datasets
if not os.path.exists("car_purchasing[1].csv"):
    files.upload()

if not os.path.exists("extended_data[1].csv"):
    files.upload()

# Load and inspect car dataset
try:
    raw_car_data = pd.read_csv("extended_data[1].csv")
    car_data = raw_car_data.drop_duplicates()
except Exception as e:
    print("Could not load car data:", e)
    car_data = pd.DataFrame()

# Define static car types
car_types = ["SUV", "Sedan", "Coupe", "Pickup", "Convertible"]

# Train model to estimate car budget
def build_model(csv_file):
    df = pd.read_csv(csv_file, encoding="ISO-8859-1")
    X = df[["age", "annual Salary", "net worth"]]
    y = df["car purchase amount"]
    reg = LinearRegression()
    reg.fit(X, y)
    return reg

# Validate necessary columns are present
def validate_car_data(df):
    expected = {"msrp", "type", "miles_per_gallon", "model"}
    if not expected.issubset(df.columns):
        missing = expected - set(df.columns)
        raise ValueError(f"Missing columns: {missing}")
    return df

# Filter cars based on user profile and predictions
def get_recommendations(model, df, age, salary, net_worth, selected_type, mpg_threshold):
    pred_input = pd.DataFrame([[age, salary, net_worth]], columns=["age", "annual Salary", "net worth"])
    budget = model.predict(pred_input)[0]
    lower, upper = budget - 1000, budget + 1000

    matches = df[
        (df["msrp"].between(lower, upper)) &
        (df["type"].str.lower() == selected_type.lower()) &
        (df["miles_per_gallon"] >= mpg_threshold)
    ]

    print(f"Estimated budget: ${budget:,.2f}")
    if matches.empty:
        print("No matches found.")
    else:
        display(matches[["model", "msrp", "type", "miles_per_gallon"]])

# Widgets for input
layout = widgets.Layout(width='500px')

age_input = widgets.FloatText(description="Age:", layout=layout)
salary_input = widgets.FloatText(description="Salary:", layout=layout)
net_input = widgets.FloatText(description="Net Worth:", layout=layout)
type_input = widgets.Dropdown(options=car_types, description="Type:", layout=layout)
mpg_input = widgets.FloatText(description="Min MPG:", layout=layout)
go_button = widgets.Button(description="Find Cars")
output = widgets.Output()

# Display form
display(Markdown("### Enter your info"))
display(age_input, salary_input, net_input, type_input, mpg_input, go_button, output)

# Handle button click
def handle_click(_):
    with output:
        output.clear_output()
        try:
            model = build_model("car_purchasing[1].csv")
            data = validate_car_data(car_data.copy())
            get_recommendations(
                model, data,
                age_input.value,
                salary_input.value,
                net_input.value,
                type_input.value,
                mpg_input.value
            )
        except Exception as e:
            print("Error:", e)

go_button.on_click(handle_click)


### Enter your info

FloatText(value=0.0, description='Age:', layout=Layout(width='500px'))

FloatText(value=0.0, description='Salary:', layout=Layout(width='500px'))

FloatText(value=0.0, description='Net Worth:', layout=Layout(width='500px'))

Dropdown(description='Type:', layout=Layout(width='500px'), options=('SUV', 'Sedan', 'Coupe', 'Pickup', 'Conve…

FloatText(value=0.0, description='Min MPG:', layout=Layout(width='500px'))

Button(description='Find Cars', style=ButtonStyle())

Output()